In [1]:
import tensorflow as tf
import numpy as np
from PIL import Image
import pandas as pd
import cv2


input_dim = 5  # one-hot size
hidden_size = 4  # output from the LSTM. 5 to directly predict one-hot
batch_size = 100   # one sentence
sequence_length = 5  # |ihello| == 6
learning_rate = 0.0001
data_mean = 119.55953717374616
data_std = 77.89011500152353

In [2]:
X = tf.placeholder(tf.float32, [None, 200, 400, 3])
Y = tf.placeholder(tf.float32, [None, 4])

with tf.variable_scope("cnn"):
    
    # conv : 5, 200, 400, 16
    conv = tf.layers.conv2d(inputs=X, filters=16, kernel_size=[4, 8], padding='SAME', activation=tf.nn.relu)
    # pool : 5, 50, 99, 16
    pool = tf.layers.max_pooling2d(inputs=conv, pool_size=[2, 4], padding='SAME', strides=1)
    # conv2 
    conv2 = tf.layers.conv2d(inputs=X, filters=64, kernel_size=[4, 8], padding='SAME', activation=tf.nn.relu)
    # pool2 
    pool2 = tf.layers.max_pooling2d(inputs=conv, pool_size=[4, 8], padding='SAME', strides=2)
    # conv3
    conv2 = tf.layers.conv2d(inputs=X, filters=128, kernel_size=[4, 8], padding='SAME', activation=tf.nn.relu)
    # pool3 
    pool3 = tf.layers.max_pooling2d(inputs=conv, pool_size=[4, 8], padding='SAME', strides=2)
    
    flat = tf.reshape(pool3, [-1, 100*200*16])
    w = tf.get_variable("w", [hidden_size, hidden_size])
    b = tf.get_variable("b", [hidden_size])
    
    logit = tf.layers.dense(inputs=flat, units=4)
    outputs = tf.sigmoid(tf.matmul(logit, w) + b)
    
    loss = -tf.reduce_mean(Y*tf.log(outputs)+(1-Y)*(tf.log(1-outputs)))
    train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

In [3]:
saver = tf.train.Saver()

with tf.Session() as sess:
    res = raw_input("Do you want addtional learning? (Name or Enter)")
    if res:
        saver.restore(sess, "./model/"+res+".ckpt")
    else:
        sess.run(tf.global_variables_initializer())
    model_name = raw_input("Model_Name: ")
    
    try:
        for epoch in range(10):
            for ims in range(1, 9):

                data_df = pd.read_csv('/mnt/d/imgs/rgb/img'+str(1)+'/log.csv', names=['name','key_out'])
                names = data_df['name'].values
                # keyouts = [8, 1, 4, 2, 15, ...]
                key_out = [eval(x) for x in data_df['key_out'].values]

                while len(names):
                    imgs = []
                    offset = min(100, len(names))
                    batch_x = names[0:offset]
                    batch_y = key_out[0:offset]
                    names = names[offset:]
                    key_out = key_out[offset:]

                    for name in batch_x:
                        img = Image.open('/mnt/d/imgs/rgb/img'+str(1)+'/img/'+name)
                        data = np.array( img, dtype='uint8')
                        data = data.reshape([200,400,3])
                        data = (data-data_mean) / data_std
                        imgs.append(data)

                    sess.run(train, feed_dict={X:np.array(imgs, np.float32), Y:batch_y})
                    print sess.run(loss, feed_dict={X:np.array(imgs, np.float32), Y:batch_y})
            if epoch == 0:
                save_path = saver.save(sess, "./model/"+model_name+"_1.ckpt")
            if epoch == 4:
                save_path = saver.save(sess, "./model/"+model_name+"_5.ckpt")
            print 'epoch', epoch+1, sess.run(loss, feed_dict={X:np.array(imgs, np.float32), Y:batch_y})
    finally:
        save_path = saver.save(sess, "./model/"+model_name+"_10.ckpt")

Do you want addtional learning? (Name or Enter)
Model_Name: full_rgb
2.0529292
2.6719162
2.1951723
2.2661824
1.428232
1.6487312
0.8760109
1.0183043
0.6467941
1.1469791
0.5757646
0.9739836
0.37097743
0.6902782
0.7301312
0.9593596
0.4325269
0.71772075
0.5508441
0.80199134
0.21559094
0.5593391
0.48009455
0.47891226
0.23545669
0.34214842
0.45150468
0.512005
0.28628692
0.4362094
0.40586364
0.49807173
epoch 1 0.49807173
0.2033981
0.32295567
0.3034655
0.37231818
0.21991877
0.33783188
0.2878212
0.26619935
0.16003066
0.31209373
0.26641113
0.2756712
0.1271008
0.2568286
0.25813174
0.2481332
0.13073042
0.24262455
0.22265862
0.23756011
0.12140455
0.22829117
0.20385367
0.19794063
0.10977686
0.22209674
0.20425992
0.18583007
0.10190231
0.19941807
0.18429832
0.17911546
epoch 2 0.17911546
0.09893901
0.18784353
0.1764855
0.1632327
0.09470939
0.18237907
0.1669263
0.15588582
0.09032396
0.17375197
0.15914002
0.14570665
0.085449256
0.17003421
0.152747
0.1376712
0.08198003
0.16068934
0.14579338
0.13217819
0.0